In [ ]:
# =============================#
# STEP 1: Install Dependencies #
# =============================#
# Check GPU
!nvidia-smi

# Install YOLOv8 (Ultralytics) + OpenCV
!pip install -q ultralytics opencv-python-headless


In [ ]:
# ===========================================#
# STEP 2: Download and Extract KITTI Dataset #
# ===========================================#
!mkdir -p kitti
!wget -q -O kitti/data_object_image_2.zip https://s3.eu-central-1.amazonaws.com/avg-kitti/data_object_image_2.zip
!wget -q -O kitti/data_object_label_2.zip https://s3.eu-central-1.amazonaws.com/avg-kitti/data_object_label_2.zip
!unzip -q kitti/data_object_image_2.zip -d kitti
!unzip -q kitti/data_object_label_2.zip -d kitti

In [ ]:
# ============================================#
# STEP 3: Convert KITTI Labels to YOLO Format #
# ============================================#
from pathlib import Path
import shutil, random

# Paths
labels_in  = Path('/content/kitti/training/label_2')
images_in  = Path('/content/kitti/training/image_2')
base_out   = Path('/content/kitti_yolo_split')

# Output folders
for split in ['train', 'val']:
    (base_out / 'images' / split).mkdir(parents=True, exist_ok=True)
    (base_out / 'labels' / split).mkdir(parents=True, exist_ok=True)

# KITTI to YOLO conversion
class_map = {
    "Car":0, "Van":1, "Truck":2, "Pedestrian":3,
    "Person_sitting":4, "Cyclist":5, "Tram":6, "Misc":7
}

IMG_W, IMG_H = 1242, 375
all_files = sorted(images_in.glob("*.png"))
random.shuffle(all_files)

n_train = int(0.8 * len(all_files))
splits = {"train": all_files[:n_train], "val": all_files[n_train:]}

for split, files in splits.items():
    for img_path in files:
        lbl_path = labels_in / img_path.with_suffix('.txt').name
        if not lbl_path.exists(): continue

        # Copy image
        shutil.copy(str(img_path), base_out / 'images' / split / img_path.name)

        # Convert label
        yolo_lines = []
        with open(lbl_path) as f:
            for line in f:
                parts = line.strip().split()
                cls = parts[0]
                if cls not in class_map: continue
                x1, y1, x2, y2 = map(float, parts[4:8])
                xc = ((x1 + x2) / 2) / IMG_W
                yc = ((y1 + y2) / 2) / IMG_H
                w = (x2 - x1) / IMG_W
                h = (y2 - y1) / IMG_H
                yolo_lines.append(f"{class_map[cls]} {xc:.6f} {yc:.6f} {w:.6f} {h:.6f}")

        if yolo_lines:
            with open(base_out / 'labels' / split / lbl_path.name, 'w') as out_f:
                out_f.write("\n".join(yolo_lines))

print("✅ Labels converted and dataset split completed.")

In [ ]:
# =========================#
# STEP 4: Create data.yaml #
# =========================#
data_yaml = """
train: /content/kitti_yolo_split/images/train
val: /content/kitti_yolo_split/images/val

nc: 8
names: ['Car', 'Van', 'Truck', 'Pedestrian', 'Person_sitting', 'Cyclist', 'Tram', 'Misc']
"""
with open("data.yaml", "w") as f:
    f.write(data_yaml.strip())

In [ ]:
# ===========================#
# STEP 5: Train YOLOv8 Model #
# ===========================#
from ultralytics import YOLO
# 1. Load a pretrained nano‑model directly
model = YOLO('yolov8n.pt')

# 2. (Optional) import any real callbacks you want
# from ultralytics.yolo.utils.callbacks import EarlyStopping

# 3. Train
results = model.train(
    data='data.yaml',                  # path to your data.yaml
    epochs=40,
    imgsz=640,
    optimizer='Adam',                  # valid optimizers: 'SGD', 'Adam', etc.
    batch=32,                          # batch size
    project='Object_Detection_for_Autonomous_Vehicles',
    name='exp1',
    # callbacks=[EarlyStopping(monitor='val/mAP50-95', patience=5)]
)




Ultralytics 8.3.135 🚀 Python-3.11.12 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=32, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=data.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=30, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8n.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=exp1, nbs=64, nms=False, opset=None, optimize=False, optimizer=Adam, overlap_mask=True, patience=100, perspective=0.0, plots=True, pose=12.0, pretrained=True, profile=False

train: Scanning /content/kitti_yolo_split/labels/train... 5984 images, 0 backgrounds, 0 corrupt: 100%|██████████| 5984/5984 [00:34<00:00, 175.94it/s]


train: New cache created: /content/kitti_yolo_split/labels/train.cache
albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01, method='weighted_average', num_output_channels=3), CLAHE(p=0.01, clip_limit=(1.0, 4.0), tile_grid_size=(8, 8))
val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 99.6±23.8 MB/s, size: 788.1 KB)


val: Scanning /content/kitti_yolo_split/labels/val... 1497 images, 0 backgrounds, 0 corrupt: 100%|██████████| 1497/1497 [00:09<00:00, 166.32it/s]


val: New cache created: /content/kitti_yolo_split/labels/val.cache
Plotting labels to Object_Detection_for_Autonomous_Vehicles/exp1/labels.jpg... 
optimizer: Adam(lr=0.01, momentum=0.937) with parameter groups 57 weight(decay=0.0), 64 weight(decay=0.0005), 63 bias(decay=0.0)
Image sizes 640 train, 640 val
Using 2 dataloader workers
Logging results to Object_Detection_for_Autonomous_Vehicles/exp1
Starting training for 30 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


...

                   all       1497       8163      0.767      0.651      0.724      0.467

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


      30/30      5.49G     0.9471     0.6185     0.9845        167        640: 100%|██████████| 187/187 [02:20<00:00,  1.33it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 24/24 [00:20<00:00,  1.14it/s]


                   all       1497       8163      0.812      0.647      0.739      0.484

30 epochs completed in 1.374 hours.
Optimizer stripped from Object_Detection_for_Autonomous_Vehicles/exp1/weights/last.pt, 6.2MB
Optimizer stripped from Object_Detection_for_Autonomous_Vehicles/exp1/weights/best.pt, 6.2MB

Validating Object_Detection_for_Autonomous_Vehicles/exp1/weights/best.pt...
Ultralytics 8.3.135 🚀 Python-3.11.12 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
Model summary (fused): 72 layers, 3,007,208 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 24/24 [00:22<00:00,  1.08it/s]


                   all       1497       8163      0.813      0.646      0.739      0.484
                   Car       1325       5698       0.92      0.826      0.921        0.7
                   Van        439        596      0.854      0.649      0.785      0.564
                 Truck        208        215      0.918      0.823      0.896      0.686
            Pedestrian        375        959      0.799      0.475      0.607      0.305
        Person_sitting         15         40      0.528      0.525      0.518      0.245
               Cyclist        234        340      0.805       0.57      0.655       0.36
                  Tram         77        117      0.806       0.78      0.858       0.55
                  Misc        160        198      0.873       0.52      0.672      0.459
Speed: 0.1ms preprocess, 0.9ms inference, 0.0ms loss, 1.8ms postprocess per image
Results saved to Object_Detection_for_Autonomous_Vehicles/exp1


In [ ]:
# ==========================================#
# STEP 6: Inference on KITTI Testing Images #
# ==========================================#
import torch
from pathlib import Path
from ultralytics import YOLO

model = YOLO('/content/best.pt')           # put the path of the best trained model
output_dir = Path("runs/detect/stream_predict")
output_dir.mkdir(parents=True, exist_ok=True)

# Stream one image at a time
for result in model.predict(
    source="/content/kitti/testing/image_2",
    stream=True,        # yield per‑image results
    imgsz=640,
    conf=0.4,
    max_det=75,
    batch=1             # keep only 1 image in memory at once
):
    # Build a target filename under our output_dir
    src_path = Path(result.path)                       # e.g. "/content/.../006111.png"
    dst_file = output_dir / src_path.name              # e.g. "runs/detect/stream_predict/006111.png"

    # Plot & save the detection to that filename
    result.plot(save=True, filename=str(dst_file))

    # free GPU memory before next image
    torch.cuda.empty_cache()

print("✅ Streaming inference complete. Check:", output_dir)



image 1/7518 /content/kitti/testing/image_2/000000.png: 160x480 (no detections), 6.8ms
image 2/7518 /content/kitti/testing/image_2/000001.png: 160x480 3 Cars, 11.9ms
image 3/7518 /content/kitti/testing/image_2/000002.png: 160x480 4 Cars, 9.1ms
image 4/7518 /content/kitti/testing/image_2/000003.png: 160x480 2 Cars, 8.5ms
image 5/7518 /content/kitti/testing/image_2/000004.png: 160x480 1 Van, 8.3ms
image 6/7518 /content/kitti/testing/image_2/000005.png: 160x480 4 Cars, 8.4ms
image 7/7518 /content/kitti/testing/image_2/000006.png: 160x480 2 Cars, 8.7ms
image 8/7518 /content/kitti/testing/image_2/000007.png: 160x480 3 Cars, 1 Tram, 12.5ms
image 9/7518 /content/kitti/testing/image_2/000008.png: 160x480 3 Cars, 8.2ms
image 10/7518 /content/kitti/testing/image_2/000009.png: 160x480 1 Pedestrian, 8.2ms
image 11/7518 /content/kitti/testing/image_2/000010.png: 160x480 3 Cars, 8.1ms
image 12/7518 /content/kitti/testing/image_2/000011.png: 160x480 (no detections), 8.3ms
image 13/7518 /content/kitt

In [ ]:
# ================================#
# STEP 7: Display Images To Check #
# ================================#
from IPython.display import Image, display
import os

pred_dir = '/content/runs/detect/stream_predict'
files    = sorted(os.listdir(pred_dir))

for fname in files[:5]:
    display(Image(os.path.join(pred_dir, fname)))
